In [1]:
import tensorflow as tf
import numpy as np
from sklearn import datasets

In [2]:
class Autoencoder:
    def __init__(self, input_dim, hidden_dim, epoch=250, learning_rate=0.001):
        self.epoch = epoch
        self.learning_rate = learning_rate
        x = tf.placeholder(dtype=tf.float32, shape=[None, input_dim])
        with tf.name_scope('encode'):
            weights = tf.Variable(tf.random_normal([input_dim, hidden_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([hidden_dim]), name='biases')
            encoded = tf.nn.tanh(tf.matmul(x, weights) + biases)
        with tf.name_scope('decode'):
            weights = tf.Variable(tf.random_normal([hidden_dim, input_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([input_dim]), name='biases')
            decoded = tf.matmul(encoded, weights) + biases
        
        self.x = x
        self.encoded = encoded
        self.decoded = decoded
        self.loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.x, self.decoded))))
        self.train_op = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss)
        self.saver = tf.train.Saver()
        
    def train(self, data):
        num_samples = len(data)
        
        with tf.Session() as sess:
            loss_summary = tf.summary.scalar("loss", self.loss)
            merged = tf.summary.merge_all()
            writer = tf.summary.FileWriter("./logs")
            writer.add_graph(sess.graph)
            sess.run(tf.global_variables_initializer())
            for i in range(self.epoch):
                for j in range(num_samples):
                    summary_str, l, _ = sess.run([merged, self.loss, self.train_op], feed_dict={self.x: [data[j]]})
                    writer.add_summary(summary_str, i)  
                if i % 10 == 0:
                    print('epoch {0}: loss = {1}'.format(i, l))
                    self.saver.save(sess, './model.ckpt') 
            self.saver.save(sess, './model.ckpt')
            
    def test(self, data): 
        with tf.Session() as sess:
            self.saver.restore(sess, './model.ckpt')
            hidden, reconstructed = sess.run([self.encoded, self.decoded], feed_dict={self.x: data})
        print('input', data)
        print('compressed', hidden)
        print('reconstructed', reconstructed)
        return reconstructed

In [3]:
hidden_dim = 1
data = datasets.load_iris().data
input_dim = len(data[0])
ae = Autoencoder(input_dim, hidden_dim)
ae.train(data)
ae.test([[8, 4, 6, 2]])

Instructions for updating:
Colocations handled automatically by placer.
epoch 0: loss = 4.16321325302124
epoch 10: loss = 2.2118756771087646
epoch 20: loss = 0.8783564567565918
epoch 30: loss = 0.48283419013023376
epoch 40: loss = 0.41726112365722656
epoch 50: loss = 0.40935632586479187
epoch 60: loss = 0.4083385467529297
epoch 70: loss = 0.40829169750213623
epoch 80: loss = 0.4084678292274475
epoch 90: loss = 0.4091434180736542
epoch 100: loss = 0.4116896986961365
epoch 110: loss = 0.41778191924095154
epoch 120: loss = 0.4105755388736725
epoch 130: loss = 0.3709574043750763
epoch 140: loss = 0.3570749759674072
epoch 150: loss = 0.35071608424186707
epoch 160: loss = 0.3419930338859558
epoch 170: loss = 0.3313843309879303
epoch 180: loss = 0.3174893260002136
epoch 190: loss = 0.3006332516670227
epoch 200: loss = 0.2828517258167267
epoch 210: loss = 0.2661285400390625
epoch 220: loss = 0.25069692730903625
epoch 230: loss = 0.23788663744926453
epoch 240: loss = 0.2296089380979538
Instruct

array([[6.6596584, 2.835511 , 5.816242 , 2.0642352]], dtype=float32)